In [2]:
import cv2
from PIL import Image, ImageDraw , ImageFont
import numpy

In [20]:
import re #텍스트파일 리스트로 변환
f = open("Tracker_log3.txt", "r")

lst = []
while True:
    line = f.readline().strip()
    if not line: 
        break
    ns = [int(float(w)) for w in line.split(' ')]
    lst.append(ns)

In [12]:
from PIL import ImageFont
fontpath = "malgun.ttf"
font = ImageFont.truetype(fontpath, 20)
#font = ImageFont.truetype("arial.ttf", font_size)  # 영어 텍스트가 지원되는 폰트로 변경하세요

In [28]:
def drawRect1(image, point1, point2, color=(0, 255, 0), thickness=1, lineType=cv2.LINE_AA):
    result = image.copy()
    return cv2.rectangle(result, point1, point2, color, thickness, lineType)

def drawRect2(image, point1, point2, color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA):
    result = image.copy()
    return cv2.rectangle(result, point1, point2, color, thickness, lineType)

def drawoverlap(image, point1,point2,point3,point4, color=(255, 0, 0)):
    result = image.copy()
    #result = cv2.rectangle(result, (point1,point2), (point3,point4), (0, 255, 0,100),cv2.FILLED, cv2.LINE_4)
    
    #cv2 -> pil로 변경
    rgb_image = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    convert_pil_image = Image.fromarray(rgb_image)
    
    #상자 그리기
    draw = ImageDraw.Draw(convert_pil_image,'RGBA')
    draw.rectangle((point1,point2,point3,point4), fill=(255,0,0,50), width=2)
    
    
    #경고창
    text = "경고:주의가 필요한 상황이 감지되었습니다"
    
    text_width, text_height = draw.textsize(text, font=font)
    text_position = ((1920 - text_width) // 2, (1028 - text_height) // 2)

    # 텍스트 주변에 하이라이트 박스 생성
    highlight_box = (
        text_position[0] - 20, text_position[1] - 30,
        text_position[0] + text_width + 20, text_position[1] + text_height +30
    )

    # 텍스트 주변에 하이라이트 박스 채우기
    draw.rectangle(highlight_box, fill="#ffff33")

    # 경고 메시지 출력
    draw.text(text_position, text, fill=(0,0,0) ,font=font)
        
    #pil -> cv2로 변경
    
    numpy_image = numpy.array(convert_pil_image)
    convert_cv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

    #return cv2.putText(result, text, (300,500), cv2.FONT_HERSHEY_DUPLEX, 3, (0, 0, 255),thickness=5,lineType= cv2.LINE_AA)
    return convert_cv_image

def imageProcessing2(image,lst):
    result = image.copy()
    pt1 = (lst[0], lst[1])
    pt2 = (lst[2], lst[3])
    pt3 = (0 , 600)
    pt4 = (1920 , 750)
    result = drawRect1(result, pt1, pt2, (0, 0, 255), 1)
    result = drawRect2(result, pt3, pt4, (255, 0, 0), 1)
    
    # 겹친 부분이 있을 때만 색칠
    overlap_area = (max(pt1[0], pt3[0]),
    max(pt1[1], pt3[1]),
    min(pt2[0], pt4[0]),
    min(pt2[1], pt4[1])
    )
    if overlap_area[0] < overlap_area[2] and overlap_area[1] < overlap_area[3]:
        result = drawoverlap(result,overlap_area[0],overlap_area[1],overlap_area[2],overlap_area[3],(255, 0, 0))
       
    return result


path = 'test_1.mp4'
cap = cv2.VideoCapture(path)
if cap.isOpened():
    print("Video Opened")
else:
    print("Video Not Opened")
    print("Program Abort")
    exit()
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("video.mp4", fourcc, fps, (width, height), True) #영상 저장
i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        output = imageProcessing2(frame,lst[i])
        out.write(output)
        #cv2.line(frame, (50, 50), (100,100), (0,0, 255),5)
        cv2.imshow("Input", output)

    else:
        break
    if cv2.waitKey(int(1000.0/fps)) & 0xFF == ord('q'):
        break
    i +=1
cap.release()
cv2.destroyAllWindows()

Video Opened


C:\Users\user\AppData\Local\Temp\ipykernel_18000\2699998111.py:25: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
C:\Users\user\AppData\Local\Temp\ipykernel_18000\2699998111.py:25: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
C:\Users\user\AppData\Local\Temp\ipykernel_18000\2699998111.py:25: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
C:\Users\user\AppData\Local\Temp\ipykernel_18000\2699998111.py:25: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, fon